In [ ]:
!pip install pyspark

### 1 - Inicialmente iremos preparar o ambiente, definindo o diretório onde nosso código será desenvolvido.
### Para este diretório iremos copiar o arquivo nomes_aleatorios.txt.

In [ ]:
#  Após, em nosso script Python, devemos importar as bibliotecas necessárias:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

In [ ]:
# Aplicando as bibliotecas do Spark, podemos definir a Spark Session e sobre ela definir o Context para habilitar o módulo SQL
spark = SparkSession \
    .builder \
    .master("local[*]")\
    .appName("Exercicio Intro") \
    .getOrCreate()

In [ ]:
# Nesta etapa, adicione código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. Carregue-o para dentro de um dataframe chamado df_nomes e, por fim,
# liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

df_nomes = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Secao_4/nomes_aleatorios.txt')

df_nomes.show(5)
type(df_nomes)

+-----------------+
|              _c0|
+-----------------+
|      Rodger Keys|
|       James Cole|
|    Vanessa Boyle|
|Harriet Berringer|
|      Ruby Oleary|
+-----------------+
only showing top 5 rows



pyspark.sql.dataframe.DataFrame

### 2 - No Python, é possível acessar uma coluna de um objeto dataframe pelo atributo (por exemplo df_nomes.nome) ou por índice (df_nomes['nome']). Enquanto a primeira forma é conveniente para a exploração de dados interativos, você deve usar o formato de índice, pois caso algum nome de coluna não esteja de acordo seu código irá falhar.

In [ ]:
# Como não informamos no momento da leitura do arquivo, o Spark não identificou o Schema por padrão e definiu todas as colunas como string.
# Para ver o Schema, use o método df_nomes.printSchema().
df_nomes.printSchema()

root
 |-- _c0: string (nullable = true)



In [ ]:
from pyspark.sql.functions import *

In [ ]:
# Renomear coluna para Nomes
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nomes')

In [ ]:
# Nesta etapa, será necessário adicionar código para renomear a coluna para Nomes, imprimir o esquema e mostrar 10 linhas do dataframe.
df_nomes.show(10)

+-----------------+
|            Nomes|
+-----------------+
|      Rodger Keys|
|       James Cole|
|    Vanessa Boyle|
|Harriet Berringer|
|      Ruby Oleary|
|     Trina Norton|
|       Joan Bates|
|      Carol Avery|
|      Page Marthe|
|       Wade Brown|
+-----------------+
only showing top 10 rows



### 3 - Ao dataframe (df_nomes), adicione nova coluna chamada Escolaridade e atribua para cada linha um dos três valores de forma aleatória: Fundamental, Medio ou Superior.

Para esta etapa, evite usar funções de iteração, como por exemplo: for, while, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

In [ ]:
from pyspark.sql.functions import rand, when

df_nomes = df_nomes.withColumn(
    "Escolaridade",
    when(rand() > 0.66, "Superior").when(rand() > 0.33, "Médio").otherwise("Fundamental"))

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|      Rodger Keys|       Médio|
|       James Cole|       Médio|
|    Vanessa Boyle|       Médio|
|Harriet Berringer|       Médio|
|      Ruby Oleary| Fundamental|
|     Trina Norton|    Superior|
|       Joan Bates|       Médio|
|      Carol Avery|    Superior|
|      Page Marthe|    Superior|
|       Wade Brown|    Superior|
+-----------------+------------+
only showing top 10 rows



### 4 - Ao dataframe (df_nomes), adicione nova coluna chamada Pais e atribua para cada linha o nome de um dos 13 países da América do Sul, de forma aleatória.

Para esta etapa, evite usar funções de iteração, como por exemplo: for, while, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

In [ ]:
from pyspark.sql.functions import udf
from random import choice

paises = ["Argentina", "Bolívia", "Brasil", "Chile", "Colômbia", "Equador", "Guiana", "Paraguai", "Peru", "Suriname", "Uruguai", "Venezuela", "Guiana Francesa"]

def random_country():
    return choice(paises)

random_country_udf = udf(random_country)

df_nomes = df_nomes.withColumn("País", random_country_udf())

In [ ]:
df_nomes.show(10)

+-----------------+------------+---------------+
|            Nomes|Escolaridade|           País|
+-----------------+------------+---------------+
|      Rodger Keys|       Médio|Guiana Francesa|
|       James Cole|       Médio|         Guiana|
|    Vanessa Boyle|       Médio|       Paraguai|
|Harriet Berringer|       Médio|          Chile|
|      Ruby Oleary| Fundamental|       Colômbia|
|     Trina Norton|    Superior|        Uruguai|
|       Joan Bates|       Médio|         Guiana|
|      Carol Avery|    Superior|      Venezuela|
|      Page Marthe|    Superior|       Paraguai|
|       Wade Brown|    Superior|        Bolívia|
+-----------------+------------+---------------+
only showing top 10 rows



### 5 - Ao dataframe (df_nomes), adicione nova coluna chamada AnoNascimento e atribua para cada linha um valor de ano entre 1945 e 2010, de forma aleatória.

In [ ]:
from pyspark.sql.functions import floor, rand

df_nomes = df_nomes.withColumn("AnoNascimento", floor(rand() * (2010 - 1945 + 1) + 1945))

In [ ]:
df_nomes.show(10)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           País|AnoNascimento|
+-----------------+------------+---------------+-------------+
|      Rodger Keys|       Médio|Guiana Francesa|         1975|
|       James Cole|       Médio|         Guiana|         1951|
|    Vanessa Boyle|       Médio|       Paraguai|         1963|
|Harriet Berringer|       Médio|          Chile|         1950|
|      Ruby Oleary| Fundamental|       Colômbia|         1976|
|     Trina Norton|    Superior|        Uruguai|         1986|
|       Joan Bates|       Médio|         Guiana|         1972|
|      Carol Avery|    Superior|      Venezuela|         2000|
|      Page Marthe|    Superior|       Paraguai|         1979|
|       Wade Brown|    Superior|        Bolívia|         1985|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



### 6 - Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século. Armazene o resultado em outro dataframe chamado df_select e mostre 10 nomes deste.

In [ ]:
df_nomes.select('Nomes', 'AnoNascimento').where('AnoNascimento >= 2000').show(10)

+------------------+-------------+
|             Nomes|AnoNascimento|
+------------------+-------------+
|       Carol Avery|         2000|
|    Kimberly Brown|         2004|
|       Jessie Jean|         2007|
|     Gina Hardnett|         2010|
|     Jackie Walker|         2002|
|       Gail Hutton|         2003|
|       Dottie Keys|         2000|
|     Paul Tarleton|         2001|
|    Roberto Moretz|         2009|
|Socorro Eisenhauer|         2008|
+------------------+-------------+
only showing top 10 rows



In [ ]:
df_select = df_nomes.select('Nomes', 'AnoNascimento').where('AnoNascimento >= 2000')
df_select.show(10)

+------------------+-------------+
|             Nomes|AnoNascimento|
+------------------+-------------+
|       Carol Avery|         2000|
|    Kimberly Brown|         2004|
|       Jessie Jean|         2007|
|     Gina Hardnett|         2010|
|     Jackie Walker|         2002|
|       Gail Hutton|         2003|
|       Dottie Keys|         2000|
|     Paul Tarleton|         2001|
|    Roberto Moretz|         2009|
|Socorro Eisenhauer|         2008|
+------------------+-------------+
only showing top 10 rows



### 7 - Usando Spark SQL repita o processo da Pergunta 6. Lembre-se que, para trabalharmos com SparkSQL, precisamos registrar uma tabela temporária e depois executar o comando SQL. Abaixo um exemplo de como executar comandos SQL com SparkSQL:

In [ ]:
df_nomes.createOrReplaceTempView ("pessoas") # Criando tabela temporária
spark.sql('select Nomes, AnoNascimento from pessoas\
          where AnoNascimento >= 2000').show(10)

+------------------+-------------+
|             Nomes|AnoNascimento|
+------------------+-------------+
|       Carol Avery|         2000|
|    Kimberly Brown|         2004|
|       Jessie Jean|         2007|
|     Gina Hardnett|         2010|
|     Jackie Walker|         2002|
|       Gail Hutton|         2003|
|       Dottie Keys|         2000|
|     Paul Tarleton|         2001|
|    Roberto Moretz|         2009|
|Socorro Eisenhauer|         2008|
+------------------+-------------+
only showing top 10 rows



### 8 - Usando o método select do Dataframe df_nomes, Conte o número de pessoas que são da geração Millennials (nascidos entre 1980 e 1994) no Dataset

In [ ]:
df_nomes.where((df_nomes['AnoNascimento'] >= 1980) & (df_nomes['AnoNascimento'] <= 1994)).count()
df_cont = df_nomes.where((df_nomes['AnoNascimento'] >= 1980) & (df_nomes['AnoNascimento'] <= 1994)).count()
print(f'O número de pessoas da geração Millennials é: {df_cont}')

O número de pessoas da geração Millennials é: 2274109


### 9 - Repita o processo da Pergunta 8 utilizando Spark SQL

In [ ]:
df_nomes.createOrReplaceTempView ("contagem_pessoas") # Criando tabela temporária
spark.sql('select count(*) as numero_pessoas from contagem_pessoas where AnoNascimento >= 1980 and AnoNascimento <= 1994').show()


+--------------+
|numero_pessoas|
+--------------+
|       2274109|
+--------------+



### 10 - Usando Spark SQL, obtenha a quantidade de pessoas de cada país para cada geração abaixo. Armazene o resultado em um novo dataframe e depois mostre todas as linhas em ordem crescente de Pais, Geração e Quantidade

- Baby Boomers – nascidos entre 1944 e 1964;

- Geração X – nascidos entre 1965 e 1979;4

- Millennials (Geração Y) – nascidos entre 1980 e 1994;

- Geração Z – nascidos entre 1995 e 2015.

In [ ]:
from pyspark.sql.functions import when

# Criar uma coluna "Geracao" com base na coluna "AnoNascimento"
df_nomes = df_nomes.withColumn("Geracao",
                   when((df_nomes["AnoNascimento"] >= 1944) & (df_nomes["AnoNascimento"] <= 1964), "Baby Boomers")
                   .when((df_nomes["AnoNascimento"] >= 1965) & (df_nomes["AnoNascimento"] <= 1979), "Geração X")
                   .when((df_nomes["AnoNascimento"] >= 1980) & (df_nomes["AnoNascimento"] <= 1994), "Millennials (Geração Y)")
                   .when((df_nomes["AnoNascimento"] >= 1995) & (df_nomes["AnoNascimento"] <= 2015), "Geração Z")
                   .otherwise("Outros"))



In [ ]:
# Agrupar por Pais e Geracao e contar a quantidade de pessoas
agrupar_contar = df_nomes.groupBy("País", "Geracao").count()
agrupar_contar.show(10)

+---------------+--------------------+------+
|           País|             Geracao| count|
+---------------+--------------------+------+
|      Venezuela|           Geração X|174055|
|        Uruguai|           Geração X|175098|
|       Colômbia|Millennials (Gera...|175161|
|       Paraguai|        Baby Boomers|232606|
|Guiana Francesa|           Geração X|174935|
|       Suriname|           Geração Z|186090|
|         Brasil|           Geração X|174743|
|       Colômbia|           Geração Z|186124|
|       Colômbia|        Baby Boomers|233214|
|       Colômbia|           Geração X|175658|
+---------------+--------------------+------+
only showing top 10 rows



In [ ]:
# Renomear a coluna "count" para "Quantidade"
renomear_count = agrupar_contar.withColumnRenamed("count", "Quantidade")
renomear_count.show(10)

+---------------+--------------------+----------+
|           País|             Geracao|Quantidade|
+---------------+--------------------+----------+
|      Venezuela|           Geração X|    174055|
|        Uruguai|           Geração X|    175098|
|       Colômbia|Millennials (Gera...|    175161|
|       Paraguai|        Baby Boomers|    232606|
|Guiana Francesa|           Geração X|    174935|
|       Suriname|           Geração Z|    186090|
|         Brasil|           Geração X|    174743|
|       Colômbia|           Geração Z|    186124|
|       Colômbia|        Baby Boomers|    233214|
|       Colômbia|           Geração X|    175658|
+---------------+--------------------+----------+
only showing top 10 rows



In [ ]:
# Ordenar os resultados por Pais, Geracao e Quantidade em ordem crescente
# Mostrar resultados
ordenar = renomear_count.orderBy("País", "Geracao", "Quantidade")
resultado = ordenar.show(10)

+---------+--------------------+----------+
|     País|             Geracao|Quantidade|
+---------+--------------------+----------+
|Argentina|        Baby Boomers|    233273|
|Argentina|           Geração X|    174933|
|Argentina|           Geração Z|    185951|
|Argentina|Millennials (Gera...|    175093|
|  Bolívia|        Baby Boomers|    232241|
|  Bolívia|           Geração X|    173737|
|  Bolívia|           Geração Z|    187002|
|  Bolívia|Millennials (Gera...|    174099|
|   Brasil|        Baby Boomers|    232864|
|   Brasil|           Geração X|    174743|
+---------+--------------------+----------+
only showing top 10 rows

